In [1]:
#Module configuration
#from py2neo import Graph #Database connection
import neomodel
import pandas as pd
import datetime

In [2]:
from neomodel import config

neo4jServerAdress = "172.19.115.240"
neo4jConnectionPort = 7687  #Connection to server bolt
user= "neo4j" 
password= "Mypassword%2023"
config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)

In [3]:
from neomodel import (config, StructuredNode, StringProperty, IntegerProperty,DateTimeProperty,
                    DateTimeFormatProperty,UniqueIdProperty, RelationshipTo,StructuredRel)

config.DATABASE_URL='bolt://' +user + ':' + password + '@' + neo4jServerAdress + ':' + str(neo4jConnectionPort)

#NIP	J_Parcours	DateDebutActe	DateFinActe	UF	Service	Phase_Parcours	Activite	Dimension_Parcours	Ref_Acte	id_Sequence	Type_Sequence	id_sejour

class FOLLOWED_BY(StructuredRel):
    Nb_Days = IntegerProperty(index=True, default=0)
    NIP=StringProperty(required=False)

class Acte(StructuredNode):
    Ref_Acte = StringProperty(required=True)
    J_Parcours=IntegerProperty(required=True)
    Duree = IntegerProperty(required=True)
    UF = StringProperty(required=True)
    #Service = StringProperty(required=True)
    #Phase_Parcours =StringProperty(required=True)
    #Activite =StringProperty(required=True)
    #Dimension_Parcours = StringProperty(required=True)
    Nombre_actes=IntegerProperty(required=False, default=1)     
    Carepath = RelationshipTo('Acte', 'FOLLOWED_BY', model=FOLLOWED_BY)

class Patient(StructuredNode):
    NIP = StringProperty(unique_index=True, required=True)
    J0 = StringProperty(required=True)
    J_Parcours = IntegerProperty(required=False, default=0)
    Carepath = RelationshipTo('Acte', 'BEGIN_CAREPATH_WITH', model=FOLLOWED_BY)

In [4]:
# Define the filepath of the import csvfile
file_path = r'..\01_Datasets\DL_Project_Dataset_Cluster2-243Patients.csv'
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=';')
# Display the first few rows of the DataFrame
df


,NIP,J_Parcours,DateDebutActe,DateFinActe,UF,Service,Phase_Parcours,Activite,Dimension_Parcours,Ref_Acte,id_Sequence,Type_Sequence,id_sejour
0,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,1001,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,2,INIT_NEW,N160017801
1,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,FTNTP2,2,INIT_NEW,N1771659
2,N201900002,0,2019-01-02 00:00:00.000,2019-01-02 00:00:00.000,8532,Médecine Nucléaire,Diagnostic,TEP,Soins,ZZQL016,2,INIT_NEW,N1771659
3,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,CS,2,INIT_NEW,N1791270
4,N201900002,37,2019-02-08 00:00:00.000,2019-02-08 00:00:00.000,9050,Radiothérapie,Consultation,Consultations,Soins,MCS,2,INIT_NEW,N1791270
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49676,N201904199,505,2020-11-13 00:00:00.000,2020-11-13 00:00:00.000,9032,Oncologie Medicale,Traitement,Ambulatoire,Soins,9032,14999,TRAIT,N2120306
49677,N201904199,505,2020-11-13 00:00:00.000,2020-11-13 00:00:00.000,9059,Oncologie Medicale,Consultation,Consultations,Soins,CS,14998,SUIVI_CT,N2120193
49678,N201904199,505,2020-11-13 00:00:00.000,2020-11-13 00:00:00.000,9059,Oncologie Medicale,Consultation,Consultations,Soins,MCS,14998,SUIVI_CT,N2120193
49679,N201904199,505,2020-11-13 00:00:00.000,2020-11-13 00:00:00.000,9059,Oncologie Medicale,Consultation,Consultations,Soins,MPC,14998,SUIVI_CT,N2120193


In [5]:
#Function to extract information from a row
date_format = '%Y-%m-%d %H:%M:%S.%f'

def extract_data(row):
    from datetime import datetime
    acte = {
        'Jp': row.get('J_Parcours', None),
        'DDA': datetime.strptime(row.get('DateDebutActe', None), date_format),
        'DDF': datetime.strptime(row.get('DateFinActe', None), date_format),
        'Ref': row.get('Ref_Acte', None),
        'NIP': row.get('NIP', None),
        'UFname': str(row.get('UF', None)),
        'service': row.get('Service', None),
        'phase_parcours': row.get('Phase_Parcours', None).replace(" ", ""),
        'activite': row.get('Activite', None),
        'dim_parcours': row.get('Dimension_Parcours', None),
        'Sequence_id': row.get('id_Sequence', None),
        'Sequence_Type': row.get('Type_Sequence', None),
        'Sejour_id': row.get('id_sejour', None)
    }

    return acte

In [6]:
#Define a query function
def myquery(query, params=None):
    from neomodel import db
    results, meta = db.cypher_query(query, params, resolve_objects=True)
    
    
    return results, meta

In [7]:
#Append to a node tuple

def myappend(alist,value):
    if not isinstance(alist, list):
        newlist=[alist]
    else:
        newlist=alist
    newlist.append(value)

    return newlist

In [8]:
#Define a function to create a new acte 
def create_acte(myActe):
    new_acte=Acte(
                    Ref_Acte = myActe['Ref'],
                    Duree = int((myActe['DDF'] - myActe['DDA']).total_seconds()),
                    J_Parcours = myActe['Jp'],
                    UF = myActe['UFname'],
                    #Service = myActe['service'],
                    #Phase_Parcours = myActe['phase_parcours'],
                    #Activite = myActe['activite'],
                    #Dimension_Parcours = myActe['dim_parcours'],
                ).save()
    return new_acte

def update_acte(myNodeActe, myNewActe):
    
    #Check if J_Parcours is th same between NodeActe and NewActe

    if myNodeActe.J_Parcours==myNewActe['Jp']:
        myNodeActe.Ref_Acte=myappend(myNodeActe.Ref_Acte,myNewActe['Ref'])
        myNodeActe.UF=myappend(myNodeActe.UF,myNewActe['UFname'])
        #myNodeActe.Service=myappend(myNodeActe.Service,myNewActe['service'])
        #myNodeActe.Phase_Parcours=myappend(myNodeActe.Phase_Parcours,myNewActe['phase_parcours'])
        #myNodeActe.Activite=myappend(myNodeActe.Activite,myNewActe['activite'])
        #myNodeActe.Dimension_Parcours=myappend(myNodeActe.Dimension_Parcours,myNewActe['dim_parcours'])
        myNodeActe.Nombre_actes=myNodeActe.Nombre_actes+1
        myNodeActe.save()
        
    else:
        raise (f"Error in Updating Node {myNodeActe.id}, the new Acte doesn't have the same J parcours !")

    return myNodeActe


def create_Startnode(myActe):
    startNode=Patient(
                    NIP = myActe['NIP'],
                    J0 = myActe['DDA']
                    ).save()
    return startNode

In [9]:
#MODELIZATION 5 : 
#NOEUDS : ACTES , DIFFERENTIATED BY PHASE PARCOURS.
#RELATIONS : 
    # START : for the first acte
    # ACTES REALIZED BY DAY

#INITIALISATION   #CLEAR THE DATABASE
query="MATCH (n) DETACH DELETE n"
myquery(query)

Start=True
output=True

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    # Extract values from the DataFrame
    myActe = extract_data(row)

    if Start:
        #Create a new acte
        new_acte=create_acte(myActe)
        Active_Node=new_acte
        
        #Create a start node
        startNode=create_Startnode(myActe)

        #LINK THE START NODE TO THE ACTES
        startNode.Carepath.connect(new_acte)
        Start=False

        #SET THE START NODE AS THE PREVIOUS NODE
        Previous_Node=startNode

    else:
        if oldActe['NIP']!=myActe['NIP']: #Check if we are dealing with the same patient
            #Create a new acte
            new_acte=create_acte(myActe)
            
            Active_Node=new_acte
            
            #Create a start node
            startNode=create_Startnode(myActe)

            #LINK THE START NODE TO THE ACTES
            startNode.Carepath.connect(new_acte)
            #SET THE START NODE AS THE PREVIOUS NODE
            Previous_Node=startNode

        else:            
            #Check if we have to change the previous node
            if myActe['Jp']!=oldActe['Jp']:
                #Create a new acte
                new_acte=create_acte(myActe)

                Previous_Node=Active_Node
                Active_Node=new_acte

                #LINK THE ACTE TO THE PREVIOUS ONE
                Previous_Node.Carepath.connect(new_acte, {'Nb_Days': myActe['Jp']-Previous_Node.J_Parcours ,'NIP': myActe['NIP'] })

            else:
                #update the active node
                new_acte=update_acte(Active_Node,myActe)
                #Active_Nodes.append(new_acte)

    #Store the acte in a variable oldActe
    oldActe=myActe


In [10]:
#List exsiting graph in the database
from neomodel import db
query = """
CALL gds.graph.list()
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
ORDER BY graphName ASC
"""
results = db.cypher_query(query)
results

([['Mygraph', 24928, 24156]], ['graphName', 'nodeCount', 'relationshipCount'])

In [11]:

#delete all existings graphs
for graph in results[0]:
    query = f"""
            CALL gds.graph.drop('{graph[0]}') YIELD graphName;
            """
    results = db.cypher_query(query)

In [12]:
#Node Embedings

#Create an in-Memory graph:
query = """
CALL gds.graph.project(
  'Mygraph',
  'Acte',
  'FOLLOWED_BY',
  {
    relationshipProperties: 'Nb_Days'
  }
)
"""
results = db.cypher_query(query)


In [13]:
# If you need to specify a number of dimension :
#CALL gds.node2vec.stream('Mygraph', {embeddingDimension: 2})
#If you don't need to specify a number of dimension:
#CALL gds.node2vec.stream('Mygraph')



#run the node2vec algorithm:
query = """
CALL gds.node2vec.stream('Mygraph')
YIELD nodeId, embedding
RETURN nodeId, embedding
"""

from neomodel import db
results = db.cypher_query(query)

nb_ebemdings_dimensions=len(results[0][0][1])

In [14]:
#convert the result variable into a dict
Dict_embedings = {}

for entry in results[0]:
    node_id = entry[0]
    embedding = entry[1]
    Dict_embedings[node_id] = {'Embedding': embedding}

In [15]:
#GET THE LIST OF PATIENT IN THE DATABASE

query="""MATCH (n:Patient) RETURN n.NIP"""
list_of_NIPs = db.cypher_query(query)

In [16]:
#list_of_NIPs[0]

In [17]:
#GET THE LIST OF NODES_ID OF THE CAREPATH OF EACH PATIENT

NIP_Carepath={}
max_Carepath_lenght=0

for i,nip in enumerate(list_of_NIPs[0],start=1):
    
    list_of_Nodes_ID=[]
    Partial_Carepath=[]

    query=f"""
        MATCH (n:Acte)-[l:FOLLOWED_BY]->(m:Acte)
        WHERE l.NIP='{nip[0]}'
        RETURN id(n) AS NodeID, id(m) AS RelatedNodeID
        ORDER BY NodeID asc
        """
    list_of_Nodes_ID = db.cypher_query(query)

    Carepath=[]
    for j,relation in enumerate(list_of_Nodes_ID[0],start=0):
        Carepath.append(relation[0])
        lastnode=relation[1]

        #Partial Carepath
        Partial_Carepath.append(Carepath.copy()) 

    #construct the list
    Carepath.append(lastnode)
    #Partial Carepath
    Partial_Carepath.append(Carepath.copy()) 


    #Add the carepath to the dict
    NIP_Carepath[i]={'NIP' : nip[0], 'Carepath' : Carepath, 'SampleCareMatrix' : Partial_Carepath}
    
    #udpate the max Carepath lenght
    if max_Carepath_lenght<len(Carepath) : max_Carepath_lenght=len(Carepath)

#NIP_Carepath


In [18]:
#Construct the SampleMatrix to a determined dimension and fill zeros at the left when dimension isn't enought

#determine the size of X (nb of columns)
#Maximum lenght of carepath in the database
nb_columns=max_Carepath_lenght

#or a definite value
#nb_columns=2

#Croop the matrix to the number of column defined
for key, value in NIP_Carepath.items():
    value['SampleCareMatrix'] = [sample[-nb_columns:] for sample in value['SampleCareMatrix']]

#Add zeros at the left when the size of the sampleCareMatrix is less than to the nb of column defined
for key, value in NIP_Carepath.items():
    for sample in value['SampleCareMatrix']:
        while len(sample) < nb_columns:
            sample.insert(0, 0)

#NIP_Carepath

In [19]:
#Dict_embedings

In [20]:
#Dict_embedings[4907]['Embedding']
#Dict_embedings[NodeId]['Embedding']

In [21]:
nb_dim=nb_ebemdings_dimensions

#Create a SampleCareTensor from the SampleCareMatrix where NodeIds are replaced by their embedings
for key, value in NIP_Carepath.items():
    value['SampleCareTensor'] = []
    for CM_Index,NodeIds_List in enumerate(value['SampleCareMatrix'],start=0):
        my_Embedings=[]
        for NodesIds_index,val_NodeId in enumerate(NodeIds_List,start=0):
            if val_NodeId==0:
                my_Embedings.append([0] * nb_dim)
            else:
                my_Embedings.append(Dict_embedings[val_NodeId]['Embedding'])
        value['SampleCareTensor'].append(my_Embedings)
        

In [22]:
import numpy as np

#nb of columns is defined in a previous cell dedicated to #Construct the SampleMatrix

G_CP_matrix = np.empty((0, nb_columns))  # Initialize an empty matrix
CP_matrix = np.empty((0, nb_columns))  # Initialize an empty matrix

for index in NIP_Carepath : 
    My_CP_Vector = NIP_Carepath[index]['SampleCareTensor']
    CP_matrix = np.empty((0, nb_columns))  # Initialize an empty matrix

    for CP_item in My_CP_Vector:
        V = []
        for embedding in CP_item:
            V.append(np.transpose(embedding))

        matrix = np.column_stack(V)
        CP_matrix = np.vstack((CP_matrix, matrix))

    G_CP_matrix = np.vstack((G_CP_matrix, CP_matrix))

In [23]:
#G_CP_matrix

In [24]:
# CREATE X, Y TRAIN AND TEST SAMPLES

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Scale the features to have values between 0 and 1
scaler = MinMaxScaler()
G_scaled = scaler.fit_transform(G_CP_matrix)


# G_CP_matrix contains features (X) and labels (Y)
X = G_scaled[:, :-1]  # Extract all columns except the last one from G_CP_matrix 
Y = G_scaled[:, -1]   # Extract the last column from G_CP_matrix

# Split data into training and testing sets for both features and labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Verify shapes of the datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (1585868, 118)
X_test shape: (396468, 118)
Y_train shape: (1585868,)
Y_test shape: (396468,)


In [25]:
from sklearn import feature_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

random_state = 42

In [26]:
#LINEAR REGRESSION
linreg = LinearRegression()
linreg.fit(X_train, Y_train)
linreg_pred = linreg.predict(X_test)
linreg_score = round(linreg.score(X_test, Y_test), 4)

In [27]:
#DECISION TREE
tree = DecisionTreeRegressor(max_depth=5, random_state=random_state)
tree.fit(X_train, Y_train)
tree_pred = tree.predict(X_test)
tree_score = round(tree.score(X_test, Y_test), 4)
print(f"Linear Regression score : {linreg_score}\nDecision Tree score : {tree_score}")

Linear Regression score : 0.9694
Decision Tree score : 0.9607


In [28]:
#RANDOMFOREST
forest = RandomForestRegressor(max_depth=5, oob_score=True, random_state=random_state)
forest.fit(X_train, Y_train)
forest_pred = forest.predict(X_test)
forest_score = round(forest.score(X_test, Y_test), 4)


print(f"Linear Regression score : {linreg_score}\nDecision Tree score : {tree_score}")
print(f"Random Forest score : {forest_score}")

In [ ]:
#RETURN SCORES
print(f"Linear Regression score : {linreg_score}\nDecision Tree score : {tree_score}")
print(f"Random Forest score : {forest_score}\n")

Linear Regression score : 0.9565
Decision Tree score : 0.948
Random Forest score : 0.9499



In [ ]:
from sklearn import metrics
def regression_metrics(y_true, y_pred, form=':.4d'):
    mse = metrics.mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = metrics.mean_absolute_error(y_true, y_pred)
    rmsle = np.sqrt(metrics.mean_squared_log_error(y_true, y_pred))
    r2 = metrics.r2_score(y_true, y_pred)
    print(f"RMSE : {rmse:.4f}\nMSE : {mse:.4f}\nMAE : {mae:.4f}\nR2 : {r2:.4f}\nRMSLE : {rmsle:.4f}")

In [ ]:

model_pred_dic = {'Linear Regression': linreg_pred,
                  'Decision Tree': tree_pred,
                  'Random Forest': forest_pred
                  }
for model, preds in model_pred_dic.items():
    print(model)
    regression_metrics(Y_test, preds)
    print('-'*30)

Linear Regression
RMSE : 0.0094
MSE : 0.0001
MAE : 0.0040
R2 : 0.9565
RMSLE : 0.0064
------------------------------
Decision Tree
RMSE : 0.0103
MSE : 0.0001
MAE : 0.0050
R2 : 0.9480
RMSLE : 0.0070
------------------------------
Random Forest
RMSE : 0.0101
MSE : 0.0001
MAE : 0.0048
R2 : 0.9499
RMSLE : 0.0069
------------------------------


In [ ]:
#Let's construct a function Vec_to_Node in order to construct the prevision of carepath.

#function Pred_2_Vec


#function Vec_2_node
def Vec_2_node(query_vector, embeddings):
    closest_node_id = None
    closest_distance = float('inf')
    
    for node_id, node_embedding in embeddings.items():
        embedding_vector = node_embedding['Embedding']
        distance = np.linalg.norm(query_vector - embedding_vector)  # Euclidean distance
        if distance < closest_distance:
            closest_distance = distance
            closest_node_id = node_id
            
    return closest_node_id

# Example usage
query_vector = np.array([0.5, 0.3, -0.1])  # Example vector you want to find the node for
closest_node_id = Vec_2_node(query_vector, Dict_embedings)


In [22]:
## ALTERNATIVE
#CONSTRUCT THE X-Y Tensor BY CONCATENATE SamplesCareTensors in rows.
import numpy as np

# Extract SampleCareTensors from the dictionary
sample_tensors = [NIP_Carepath[key]['SampleCareTensor'] for key in NIP_Carepath]
# Concatenate the tensors along the first axis
XY_tensor = np.concatenate(sample_tensors, axis=0)

In [23]:
XY_tensor.shape

(8, 4, 128)

In [ ]:
#FIND THE SHORTEST PATH

#CREATE A SHORTEST PATH GRAPH
query = """
CALL gds.graph.project(
    'myGraph2',
    'Acte',
    'FOLLOWED_BY',
    {
        nodeProperties: ['J_Parcours'],
        relationshipProperties: 'Nb_Days'
    }
)
"""
results = db.cypher_query(query)

In [ ]:
#RUN THE ALGORITHM

query = """
MATCH (source:Acte {Ref_Acte: 'CS'}), (target:Acte {Phase_Parcours: 'Traitement'})
CALL gds.shortestPath.astar.stream('myGraph2', {
    sourceNode: source,
    targetNode: target,
    JParcours_Property: 'J_Parcours',
    relationshipWeightProperty: 'FOLLOWED_BY'
})
YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
RETURN
    index,
    gds.util.asNode(sourceNode).name AS sourceNodeName,
    gds.util.asNode(targetNode).name AS targetNodeName,
    totalCost,
    [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodeNames,
    costs,
    nodes(path) as path
ORDER BY index
"""

from neomodel import db
results, meta = db.cypher_query(query, None, resolve_objects=True)

ClientError: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.shortestPath.astar.stream`: Caused by: java.lang.IllegalArgumentException: Multiple errors in configuration arguments:
				No value specified for the mandatory configuration parameter `longitudeProperty`
				No value specified for the mandatory configuration parameter `latitudeProperty`}